In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle as pk
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
dataseta = pd.read_csv('Train.tsv',sep='\t',dtype=object)
dataset_test1 = pd.read_csv('Test.tsv',sep='\t',dtype=object)
dataseta.drop(dataseta[dataseta['Payer']=='*'].index,inplace=True)
dataseta.drop(columns='DaystoPay',inplace=True)
dataseta.info()
dataseta.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88857 entries, 134 to 88990
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Payer               88857 non-null  object
 1   Debit_CreditInd.    88857 non-null  object
 2   PercentofPayAmount  88857 non-null  object
dtypes: object(3)
memory usage: 2.7+ MB


,Payer,Debit_CreditInd.,PercentofPayAmount
count,88857,88857,88857
unique,447,2,34010
top,301088,S,100
freq,25137,81277,13891


In [2]:
Convert_dict = {'Payer': str, 
                'Debit_CreditInd.': str,
                'PercentofPayAmount': float
               } 
  
dataseta = dataseta.astype(Convert_dict) 
print(dataseta.dtypes) 
dataseti=dataseta.groupby(['Payer','Debit_CreditInd.'])['PercentofPayAmount'].mean()
datasetii = pd.DataFrame(dataseti).reset_index()
inner_join = pd.merge(dataseta,  
                      datasetii,  
                      on =['Payer','Debit_CreditInd.'],  
                      how ='inner') 
inner_join['rank'] = inner_join.groupby(['Payer','Debit_CreditInd.'])['PercentofPayAmount_x'].rank()
dataset_ready=inner_join[['Payer','Debit_CreditInd.','rank','PercentofPayAmount_x']]
dataset=dataset_ready
Convert_dict1 = {'Payer': str, 
                'Debit_CreditInd.': str,
                'rank':int,
                'PercentofPayAmount_x': str
               } 
  
dataset = dataset.astype(Convert_dict1)
#dataset=dataset.head(50000)
# Feature Scaling

X1=dataset[['Payer','Debit_CreditInd.','rank']]
dataset_ready.head()

dataset_test=dataset_test1
y = dataset.iloc[:, -1].values
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False,handle_unknown='ignore'), [0,1])], remainder='passthrough')
X = np.array(ct.fit_transform(X1))
'''
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

from sklearn.ensemble import RandomForestClassifier
classifier1 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0,bootstrap=True,
                                    ccp_alpha=0.0, class_weight=None,
                        max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       n_jobs=None, oob_score=False, verbose=0,
                       warm_start=False)
'''
from sklearn.neighbors import KNeighborsClassifier
classifier1 = KNeighborsClassifier(n_neighbors = 20,weights='uniform', metric = 'minkowski', p = 2)
classifier1.fit(X, y)

Payer                  object
Debit_CreditInd.       object
PercentofPayAmount    float64
dtype: object


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                     weights='uniform')

In [3]:
dataset_test['rank']=1
X_test_encoded=np.array(ct.transform(dataset_test))
y_predict=classifier1.predict(X_test_encoded)
dataset_test['PercentofPayAmount']=y_predict
print(dataset_test)
y_pred=classifier1.predict(X)
'''
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import average_precision_score,classification_report
from sklearn.metrics import f1_score
#average_precision = average_precision_score(y_test_in, y_pred)
cm = confusion_matrix(y,y_pred)
print( accuracy_score(y,y_pred))
print(classification_report(y,y_pred))
'''

       Payer Debit_CreditInd.  rank PercentofPayAmount
0     100030                H     1              100.0
1     100030                S     1              100.0
2   70000010                S     1        2.298850575
3     300767                S     1        2.298850575
4     100040                H     1              100.0
5     100059                H     1              100.0
6     301096                S     1              100.0
7     301097                S     1        2.298850575
8     301136                S     1        2.298850575
9   70000000                S     1        2.298850575
10    300529                H     1              100.0


'\n# Making the Confusion Matrix\nfrom sklearn.metrics import confusion_matrix, accuracy_score\nfrom sklearn.metrics import average_precision_score,classification_report\nfrom sklearn.metrics import f1_score\n#average_precision = average_precision_score(y_test_in, y_pred)\ncm = confusion_matrix(y,y_pred)\nprint( accuracy_score(y,y_pred))\nprint(classification_report(y,y_pred))\n'

In [ ]:

'''
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0,bootstrap=True,
                                    ccp_alpha=0.0, class_weight=None,
                        max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       n_jobs=None, oob_score=False, verbose=0,
                       warm_start=False)

classifier2.fit(X, y)
dataset_test['rank']=1
X_test_encoded=np.array(ct.transform(dataset_test))
y_predict=classifier2.predict(X_test_encoded)
dataset_test['PercentofPayAmount']=y_predict
print(dataset_test)
y_pred=classifier2.predict(X)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import average_precision_score,classification_report
from sklearn.metrics import f1_score
#average_precision = average_precision_score(y_test_in, y_pred)
cm = confusion_matrix(y,y_pred)
print( accuracy_score(y,y_pred))
print(classification_report(y,y_pred))
'''

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)